In [ ]:
import tensorflow as tf
print(tf.__version__)

# Problem Statement

# Data Preprocessing

In [7]:
import tensorflow as tf
from tensorflow import keras

train = tf.keras.utils.image_dataset_from_directory('CNN Data/train', labels = 'inferred')
val = tf.keras.utils.image_dataset_from_directory('CNN Data/val', labels = 'inferred')

Found 9999 files belonging to 10 classes.
Found 2000 files belonging to 10 classes.


# Deep Learning Model: CNN

In [8]:
types = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    #tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),

    tf.keras.layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(),
    #tf.keras.layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
    #tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)#, activation=tf.nn.softmax)
])

In [10]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.75):
      print("Cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [11]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
model.fit(train, validation_data=val, epochs=60, callbacks=[callbacks])

Epoch 1/60


313/313 [==============================] - 120s 380ms/step - loss: 2.3189 - accuracy: 0.0951 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/60
313/313 [==============================] - 128s 407ms/step - loss: 2.3029 - accuracy: 0.0958 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/60
313/313 [==============================] - 147s 468ms/step - loss: 2.3034 - accuracy: 0.0949 - val_loss: 2.3031 - val_accuracy: 0.1000
Epoch 4/60
 95/313 [========>.....................] - ETA: 1:19 - loss: 2.3031 - accuracy: 0.1026

# Evaluation of Model

In [ ]:
model.evaluate(val)

63/63 [==============================] - 3s 49ms/step - loss: 2.1981 - accuracy: 0.2265


[2.198082447052002, 0.226500004529953]

# Hyperparameter Tuning